In [ ]:
import concurrent.futures
import os
import sys
from functools import partial

sys.path.append('..')

import matplotlib.pyplot as plt
import numpy as np

from materials import get_materials_Cvec
from safe_module import distance
from themes import colors
from themes import symmetry_classes

In [ ]:
# Analysis for Brown et al. 2016 Table 2 materials

workers = os.cpu_count()

materials = ['An0', 'An25', 'An37', 'An48', 'An60', 'An67', 'An78', 'An96']
anorthite_percentages = [0, 25, 37, 48, 60, 67, 78, 96]
data_size = 5

n_materials = len(materials)
n_symmetry_classes = len(symmetry_classes)

In [ ]:
vecs = (get_materials_Cvec(material="Brown"))

c_vecs_Brown = [vecs[i] for i in np.arange(0,15,2)]
stdv_vecs_Brown = [vecs[i] for i in np.arange(1,16,2)]

In [ ]:
# Section to find beta values for realizations of the Brown2016 materials

means_Brown = np.empty((n_symmetry_classes, n_materials))
stdvs_Brown = np.empty((n_symmetry_classes, n_materials))

for i in range(n_materials):
    
    print(f'running analysis for material {materials[i]}')
    
    with open(f'output_{materials[i]}.txt', 'w') as f:
        f.write('C11 \tC12 \tC13 \tC14 \tC15 \tC16 \tC22 \tC23 \tC24 \tC25 \tC26 \tC33 \tC34 \tC35 \tC36 \tC44 '
         '\tC45 \tC46 \tC55 \tC56 \tC66\n')

    c_vecs = []

    for _ in range(data_size):
        c_vec_Brown_random = np.around( np.random.normal( c_vecs_Brown[i], stdv_vecs_Brown[i]), 1 )
        with open(f'output_{materials[i]}.txt', 'a') as f:
            for value in c_vec_Brown_random:
                f.write(f'{value:.1f} \t')
            f.write('\n')
        c_vecs.append(c_vec_Brown_random)
        
    betas = np.empty((n_symmetry_classes, data_size))

    distance_new = partial(distance, use_parallel_processing=False)

    for j, sigma in enumerate(symmetry_classes):   
        print(f'running minimizer for symmetry class: {sigma}')
        with concurrent.futures.ProcessPoolExecutor(max_workers=workers) as executor:
            results = list(executor.map(distance_new, c_vecs, [sigma]*data_size))
        for k in range(data_size):
            betas[j,k] = results[k][0]
    
        means_Brown[j,i] = np.mean(betas[j,:])
        stdvs_Brown[j,i] = np.std(betas[j,:])
        
    np.save(f'betas_brown_{materials[i]}', betas)

In [ ]:
# Section to analyze saved beta values

means_Brown = np.empty((n_symmetry_classes, n_materials))
stdvs_Brown = np.empty((n_symmetry_classes, n_materials))

for i in range(n_materials):
    
    betas = np.load(f'betas_brown_{materials[i]}.npy')

    for j in range(n_symmetry_classes):   
        means_Brown[j,i] = np.mean(betas[j,:])
        stdvs_Brown[j,i] = np.std(betas[j,:])

In [ ]:
plt.figure(figsize=(7,10))

x = np.arange(n_symmetry_classes)

clrs = ['red', 'orange', 'teal', 'cyan', 'blue', 'purple', 'grey', 'black']

for i, material in enumerate(materials):
    plt.plot(x, means_Brown[:,i], '-o', label=material, color=clrs[i])
    plt.errorbar(x, means_Brown[:,i], yerr=2*stdvs_Brown[:,i], color=clrs[i], capsize=5, capthick=1)

plt.xticks(x, symmetry_classes, fontsize=15)
plt.yticks(fontsize=15)
plt.legend(loc=1, fontsize=13, title='(Brown2016)', title_fontsize=15)
plt.grid()
plt.xlabel(r'${\Sigma}$', fontsize=20)
plt.ylabel(r'$\beta_{\Sigma}$ (°)', fontsize=20)
plt.savefig('Brown_Table2_1.png')

In [ ]:
plt.figure(figsize=(7,10))

x = anorthite_percentages

for i, symmetry_class in enumerate(symmetry_classes):
    plt.plot(x, means_Brown[i], '-o', color=colors[symmetry_class], label=symmetry_class)
    plt.errorbar(x, means_Brown[i], yerr=2*stdvs_Brown[i], color=colors[symmetry_class], capsize=5, capthick=1)

plt.xticks(np.arange(0,101,10), fontsize=15)
plt.yticks(fontsize=15)
plt.legend(loc=1, fontsize=13, title='${\Sigma}$ (Brown2016)', title_fontsize=15)
plt.grid()
plt.xlabel('Anorthite percentage (%)', fontsize=20)
plt.ylabel(r'$\beta_{\Sigma}$ (°)', fontsize=20)
plt.savefig('Brown_Table2_2.png')